# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [15]:
# import libraries
import pandas as pd
import pandas as pd
import numpy as np
import os
import pickle
import nltk
import re
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

In [16]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/nerav/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/nerav/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/nerav/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/nerav/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [18]:
# load data from database
engine = create_engine('sqlite:////Users/nerav/Documents/GitHub/DSND_Term2/project_files/disaster/data/DisasterResponse.db')
df = pd.read_sql_table('disaster_message',engine)
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [19]:
def tokenize(text):
    """Tokenization function. Receives as input raw text which afterwards normalized, stop words removed, stemmed and lemmatized.
    Returns tokenized text"""
    
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    stop_words = stopwords.words("english")
    
    #tokenize
    words = word_tokenize (text)
    
    #stemming
    stemmed = [PorterStemmer().stem(w) for w in words]
    
    #lemmatizing
    words_lemmed = [WordNetLemmatizer().lemmatize(w) for w in stemmed if w not in stop_words]
   
    return words_lemmed

In [20]:
#Test the function
text = "BRIAR PATH & WHITEMARSH LN; HATFIELD TOWNSHIP; Station 345; 2015-12-10 @ 17:29:21;"

tokenize(text)

['briar',
 'path',
 'whitemarsh',
 'ln',
 'hatfield',
 'township',
 'station',
 '345',
 '2015',
 '12',
 '10',
 '17',
 '29',
 '21']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [21]:
# Pipiline simple version
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize))
    , ('tfidf', TfidfTransformer())
    , ('clf', MultiOutputClassifier(RandomForestClassifier()))])

pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf_

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [23]:
%%time
# train classifier
pipeline.fit(X_train, Y_train)

CPU times: user 5min 20s, sys: 1.91 s, total: 5min 22s
Wall time: 5min 23s


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

In [24]:
# predict on test data
Y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [25]:
y_pred_pd = pd.DataFrame(Y_pred, columns = Y_test.columns)
for column in Y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(Y_test[column],y_pred_pd[column]))

------------------------------------------------------

FEATURE: related

              precision    recall  f1-score   support

           0       0.72      0.41      0.52      1266
           1       0.83      0.95      0.88      3938
           2       0.59      0.42      0.49        40

    accuracy                           0.81      5244
   macro avg       0.71      0.59      0.63      5244
weighted avg       0.80      0.81      0.79      5244

------------------------------------------------------

FEATURE: request

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4349
           1       0.85      0.48      0.61       895

    accuracy                           0.90      5244
   macro avg       0.88      0.73      0.78      5244
weighted avg       0.89      0.90      0.88      5244

------------------------------------------------------

FEATURE: offer

              precision    recall  f1-score   support

           0     

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.90      0.95      0.92      3806
           1       0.84      0.71      0.77      1438

    accuracy                           0.88      5244
   macro avg       0.87      0.83      0.85      5244
weighted avg       0.88      0.88      0.88      5244

------------------------------------------------------

FEATURE: floods

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      4833
           1       0.90      0.52      0.66       411

    accuracy                           0.96      5244
   macro avg       0.93      0.76      0.82      5244
weighted avg       0.96      0.96      0.95      5244

------------------------------------------------------

FEATURE: storm

              precision    recall  f1-score   support

           0       0.95      0.99      0.97      4758
           1       0.78      0.52      0.63       486

    accuracy                         

### 6. Improve your model
Use grid search to find better parameters. 

In [26]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x7ff696bac3b0>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=No

In [27]:
%%time
parameters = {
    #'vect__ngram_range': ((1, 1), (1, 2)),
    #'vect__max_df': (0.5, 0.75, 1.0),
    #'tfidf__use_idf': (True, False), 
    #'clf__estimator__n_estimators': [50, 100, 200],
     'clf__estimator__min_samples_split': [2, 3, 4]
}

pipeline_cv = GridSearchCV(pipeline, param_grid = parameters,verbose = 3)

CPU times: user 42 µs, sys: 1 µs, total: 43 µs
Wall time: 48.2 µs


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [28]:
%%time
pipeline_cv.fit(X_train, Y_train)
print('Best Parameters:', pipeline_cv.best_params_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] clf__estimator__min_samples_split=2 .............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] . clf__estimator__min_samples_split=2, score=0.258, total= 4.0min
[CV] clf__estimator__min_samples_split=2 .............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.0min remaining:    0.0s


[CV] . clf__estimator__min_samples_split=2, score=0.255, total= 4.0min
[CV] clf__estimator__min_samples_split=2 .............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  7.9min remaining:    0.0s


[CV] . clf__estimator__min_samples_split=2, score=0.275, total= 3.9min
[CV] clf__estimator__min_samples_split=2 .............................
[CV] . clf__estimator__min_samples_split=2, score=0.259, total= 4.0min
[CV] clf__estimator__min_samples_split=2 .............................
[CV] . clf__estimator__min_samples_split=2, score=0.275, total= 4.0min
[CV] clf__estimator__min_samples_split=3 .............................
[CV] . clf__estimator__min_samples_split=3, score=0.256, total= 3.5min
[CV] clf__estimator__min_samples_split=3 .............................
[CV] . clf__estimator__min_samples_split=3, score=0.260, total= 3.6min
[CV] clf__estimator__min_samples_split=3 .............................
[CV] . clf__estimator__min_samples_split=3, score=0.271, total= 3.6min
[CV] clf__estimator__min_samples_split=3 .............................
[CV] . clf__estimator__min_samples_split=3, score=0.265, total= 3.6min
[CV] clf__estimator__min_samples_split=3 .............................
[CV] .

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed: 54.3min finished


Best Parameters: {'clf__estimator__min_samples_split': 3}
CPU times: user 58min 30s, sys: 27.7 s, total: 58min 58s
Wall time: 59min 2s


In [29]:
Y_pred = pipeline_cv.predict(X_test)

In [30]:
y_pred_pd = pd.DataFrame(Y_pred, columns = Y_test.columns)
for column in Y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(Y_test[column],y_pred_pd[column]))

------------------------------------------------------

FEATURE: related

              precision    recall  f1-score   support

           0       0.72      0.41      0.53      1266
           1       0.83      0.95      0.88      3938
           2       0.65      0.33      0.43        40

    accuracy                           0.81      5244
   macro avg       0.73      0.56      0.61      5244
weighted avg       0.80      0.81      0.79      5244

------------------------------------------------------

FEATURE: request

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4349
           1       0.83      0.50      0.62       895

    accuracy                           0.90      5244
   macro avg       0.87      0.74      0.78      5244
weighted avg       0.89      0.90      0.89      5244

------------------------------------------------------

FEATURE: offer

              precision    recall  f1-score   support

           0     

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.96      1.00      0.98      4833
           1       0.90      0.48      0.63       411

    accuracy                           0.96      5244
   macro avg       0.93      0.74      0.80      5244
weighted avg       0.95      0.96      0.95      5244

------------------------------------------------------

FEATURE: storm

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      4758
           1       0.76      0.57      0.65       486

    accuracy                           0.94      5244
   macro avg       0.86      0.77      0.81      5244
weighted avg       0.94      0.94      0.94      5244

------------------------------------------------------

FEATURE: fire

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5191
           1       0.00      0.00      0.00        53

    accuracy                           

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [31]:
from sklearn.base import BaseEstimator, TransformerMixin
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [32]:
from sklearn.tree import DecisionTreeClassifier

In [33]:
new_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(
        AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1, class_weight='balanced'))
    ))
])

In [34]:
%%time
new_pipeline.fit(X_train, Y_train)

CPU times: user 1min 17s, sys: 3.46 s, total: 1min 21s
Wall time: 1min 21s


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                    base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                                                                          class_weight='balanced',
                                                                                   

In [35]:
Y_pred = new_pipeline.predict(X_test)

In [36]:
y_pred_pd = pd.DataFrame(Y_pred, columns = Y_test.columns)
for column in Y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(Y_test[column],y_pred_pd[column]))

------------------------------------------------------

FEATURE: related

              precision    recall  f1-score   support

           0       0.36      0.83      0.50      1266
           1       0.92      0.51      0.65      3938
           2       0.09      0.40      0.15        40

    accuracy                           0.58      5244
   macro avg       0.46      0.58      0.43      5244
weighted avg       0.78      0.58      0.61      5244

------------------------------------------------------

FEATURE: request

              precision    recall  f1-score   support

           0       0.94      0.85      0.89      4349
           1       0.51      0.73      0.60       895

    accuracy                           0.83      5244
   macro avg       0.72      0.79      0.75      5244
weighted avg       0.87      0.83      0.84      5244

------------------------------------------------------

FEATURE: offer

              precision    recall  f1-score   support

           0     

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      3806
           1       0.78      0.79      0.78      1438

    accuracy                           0.88      5244
   macro avg       0.85      0.85      0.85      5244
weighted avg       0.88      0.88      0.88      5244

------------------------------------------------------

FEATURE: floods

              precision    recall  f1-score   support

           0       0.98      0.94      0.96      4833
           1       0.50      0.72      0.59       411

    accuracy                           0.92      5244
   macro avg       0.74      0.83      0.77      5244
weighted avg       0.94      0.92      0.93      5244

------------------------------------------------------

FEATURE: storm

              precision    recall  f1-score   support

           0       0.98      0.94      0.96      4758
           1       0.59      0.85      0.70       486

    accuracy                         

### 9. Export your model as a pickle file

In [37]:
pickle.dump(new_pipeline, open('disaster_ada_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.